# Vectorization with NumPy and SciPy

While attractive for its simplicity and flexibility, Python is not particularly efficient when it comes to speed of execution, especially when working with large amounts of data.
However, there are some libraries and built-in functions that allow to circumvent this problem.
In this tutorial, we will see how to speed up Python code through vectorization with NumPy and SciPy.
This technique will be particularly useful in your homeworks, and should be used whenever possible.

You can run this notebook in the conda environment `gc_course_env`.

## Scalar addition

In this first simple example, we want to compute the average values of two ordered sets of scalars `a_1` and `a_2` with the same number of elements *n*.
As first implementation, we can store the points in two Python lists, and define a function that iterates over the lists with a for loop:

In [1]:
def average(a_1, a_2):
    mid_points = []
    for i in range(len(a_1)):
        mid_point = (a_1[i] + a_2[i]) / 2
        mid_points.append(mid_point)
    return mid_points

To avoid the for loop, we can vectorize this operation by storing the elements iterated by the for loop along an axis of a NumPy array.
In this case, we create two arrays of shape (*n*,).
We can then perform the points average operation through element-wise summation between the two vectors:

In [2]:
def average_vectorized(a_1, a_2):
    return (a_1 + a_2) / 2

NumPy arrays can be used as input of the function `average` as well, and can be iterated as lists along the first axis.
Let us test now the efficiency of these two funcions:

In [3]:
import numpy as np

n = 10000

a_1 = np.arange(n)
a_2 = np.arange(n)

print('for loop timing:')
%timeit average(a_1, a_2)

print('vectorized timing:')
%timeit average_vectorized(a_1, a_2)

for loop timing:
3.55 ms Â± 49.3 Âµs per loop (mean Â± std. dev. of 7 runs, 100 loops each)
vectorized timing:
17.9 Âµs Â± 87.3 ns per loop (mean Â± std. dev. of 7 runs, 100000 loops each)


In our test, the vectorization speedup is approximately 200 times (this value may vary on different machines). This means that a task that requires 3 hours of computation with for loops can be accomplished in less that one minute with a properly vectorized code!

Note that the same functions can be used to perform an element-wise average of two ordered sets of vectors.
In this case, we can store the vectors in two arrays `v_1` and `v_2` with shape (*n*, 3).
We can then compare the run times of the two functions:

In [4]:
n = 100000

v_1 = np.random.random((n, 3))
v_2 = np.random.random((n, 3))

print('for loop timing:')
%timeit average(v_1, v_2)

print('vectorized timing:')
%timeit average_vectorized(v_1, v_2)

for loop timing:
157 ms Â± 2.11 ms per loop (mean Â± std. dev. of 7 runs, 10 loops each)
vectorized timing:
975 Âµs Â± 26.2 Âµs per loop (mean Â± std. dev. of 7 runs, 1000 loops each)


## Vectorization of mesh face loops

Let us consider now a triangular mesh, with vertices coordinates and faces stored, respectively, in the arrays `V` and `F`.
We want now to compute, for instance, the three vectors that bound each mesh face that we denote as `e_1`, `e_2`, and `e_3`.
For that, we can write a function that iterates over faces:

In [5]:
def face_vectors(V, F):
    e_1 = []
    e_2 = []
    e_3 = []
    for face in F:
        e_1.append(V[face[1]] - V[face[0]])
        e_2.append(V[face[2]] - V[face[1]])
        e_3.append(V[face[0]] - V[face[2]])
    return e_1, e_2, e_3

We want now to vectorize the loop over the faces.
As first, note that we can extract the array containing the indices of the first vertex of each face by taking the first column of the array `F`.
In NumPy, this can be done through [indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html) as `F[:, 0]`.
Similarly, with `F[:, 1]` and `F[:, 2]` we can extract the indices of the second and third vertex.

As a simple example, let us consider the mesh:

In [6]:
V = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0]])
F = np.array([[0, 1, 2], [2, 3, 0]])

and then we have

In [7]:
F[:, 0], F[:, 1], F[:, 2]

(array([0, 2]), array([1, 3]), array([2, 0]))

We can now extract the three arrays containing the coordinates of the three face vertices as follows:

In [8]:
V[F[:, 0]], V[F[:, 1]], V[F[:, 2]]

(array([[0, 0, 0],
        [1, 1, 0]]),
 array([[1, 0, 0],
        [0, 1, 0]]),
 array([[1, 1, 0],
        [0, 0, 0]]))

We can then rewrite our function in a vectorized way as

In [9]:
def face_vectors_vectorized(V, F):
    e_1 = V[F[:, 1]] - V[F[:, 0]]
    e_2 = V[F[:, 2]] - V[F[:, 1]]
    e_3 = V[F[:, 0]] - V[F[:, 2]]
    return e_1, e_2, e_3

To test these implementations, we first define a function that builds a triangular mesh grid with *n* x *n* vertices as follows:

In [10]:
def triangular_grid(n):
    G = np.arange(n * n).reshape(n, n)
    v1 = G[:-1, :-1].flatten()
    v2 = G[:-1, 1:].flatten()
    v3 = G[1:, 1:].flatten()
    v4 = G[1:, :-1].flatten()
    F1 = np.column_stack((v1, v3, v4))
    F2 = np.column_stack((v1, v2, v3))
    F = np.vstack((F1, F2))
    Vx, Vy = np.meshgrid(np.linspace(0, 1, n), np.linspace(0, 1, n))
    V = np.column_stack((Vx.flatten(), Vy.flatten(), np.zeros(n * n)))
    return V, F

We can now time the two functions:

In [11]:
n = 100

V, F = triangular_grid(n)

print('for loop timing:')
%timeit face_vectors(V, F)

print('vectorized timing:')
%timeit face_vectors_vectorized(V, F)

for loop timing:
60.4 ms Â± 193 Âµs per loop (mean Â± std. dev. of 7 runs, 10 loops each)
vectorized timing:
1.32 ms Â± 17.7 Âµs per loop (mean Â± std. dev. of 7 runs, 1000 loops each)


## Vectorization of face-vertices sums

Let us now imagine to have, on a mesh, some scalar values defined for each face. We want then to sum at vertices the values of each incident face. If face scalars are collected in the vector `values` of shape (#`F`,), we can compute the sums at vertices with a for loop:

In [12]:
def vertex_faces_sum(values, F):
    vertex_sum = np.zeros(np.max(F) + 1) # initializes all sums to zero. max(F) + 1 gives the total number of vertices.
    for f in range(len(F)):
        for v in F[f]:
            vertex_sum[v] += values[f]
    return vertex_sum        

Note that if face values are all equal to one,  this function returns the number of incident faces at each vertex.

To vectorize this function, a strategy is to build a matrix `M` of shape (#`V`, #`F`) with row index `i` corresponding to vertices and column index `j` corresponding to faces, place in position `i,j` the scalar belonging to face `j` if this face is incident to vertex `i` or zero otherwise, and sum along rows.
In this way, we get an array of shape (#`V`,) with vertex sums in `i` position.

Since on a mesh few faces are incident to each vertex, the matrix `M` will contain mostly zeros.
An efficient way to deal with such data is through sparse matrices which store only non-zero entries.
SciPy provides several data structures for sparse matrices in Python.
In this example, we will use a matrix in [coordinate format](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html#scipy.sparse.coo_matrix) that can be constructed by providing the array `values` containing non-zero entries, the arrays `i`, `j` with the corresponding indices, and the matrix shape.

This function can be implemented as follows:

In [13]:
from scipy import sparse

def vertex_faces_sum_vectorized(values, F):
    i = F.flatten('F')
    j = np.arange(len(F))
    j = np.tile(j, 3)
    M = sparse.coo_matrix((values[j], (i, j)), (np.max(F) + 1, len(F)))
    vertex_sum = np.array(M.sum(axis=1)).flatten()
    return vertex_sum

We can test now the corresponding execution times:

In [14]:
values = np.ones(len(F))

print('for loop timing:')
%timeit vertex_faces_sum(values, F)

print('vectorized timing:')
%timeit vertex_faces_sum_vectorized(values, F)

for loop timing:
32.6 ms Â± 311 Âµs per loop (mean Â± std. dev. of 7 runs, 10 loops each)
vectorized timing:
490 Âµs Â± 7.91 Âµs per loop (mean Â± std. dev. of 7 runs, 1000 loops each)


In our test, we found a speed up of approximately 50 times. This is still a significant improvement.

## Example: vectorized mesh area gradient

We want now to compute the gradient of the mesh area $A_\Omega$ with respect to the $x$ coordinate of a vertex $v$, denoted as $x_v$.
Since a change of $x_v$ will influence only the area of its incident faces $f\sim v$, the gradient is given by
$$\frac{\partial A_{\Omega}}{\partial x_v} = \sum_{f\sim v} \frac{\partial A_f}{\partial x_v}$$
where $A_f$ is the area of the face $f$.
In this case, we have to sum, at each vertex, the scalars $\frac{\partial A_f}{\partial x_v}$ defined at face vertices.

For a triangular mesh, we can collect these values in an array of shape (#`F`, 3), where each column contains the derivative of $A_f$ with respect to one of its three vertices.
To perform the sum, we can define a function that loops over faces:

In [15]:
def vertex_faces_gradients(dAx_f, F):
    dAx = np.zeros(np.max(F) + 1)
    for f in range(len(F)):
        for v in range(len(F[f])):
            dAx[F[f, v]] += dAx_f[f, v]
    return dAx  

To vectorize this operation, we can sligthly modify our function `vertex_faces_sum_vectorized` as follows:

In [16]:
def vertex_faces_gradients_vectorized(dAx_f, F):
    i = F.flatten('F')
    j = np.arange(len(F))
    j = np.tile(j, 3)
    M = sparse.coo_matrix((dAx_f.flatten('F'), (i, j)), (np.max(F) + 1, len(F)))
    dAx = np.array(M.sum(axis=1)).flatten()
    return dAx

and we can test the execution times of these two functions:

In [17]:
dAx_f = np.ones((len(F), 3))

print('for loop timing:')
%timeit vertex_faces_gradients(dAx_f, F)

print('vectorized timing:')
%timeit vertex_faces_gradients_vectorized(dAx_f, F)

for loop timing:
32.5 ms Â± 261 Âµs per loop (mean Â± std. dev. of 7 runs, 10 loops each)
vectorized timing:
391 Âµs Â± 5.53 Âµs per loop (mean Â± std. dev. of 7 runs, 1000 loops each)


## Vectorization of cells-vertices sums

Consider now to sum at vertices quantities from incident edges or from incident edge vertices (as for instance the gradient of edge lengths with respect to its two vertices).
For that, we can use a function similar to `vertex_faces_gradients_vectorized` defined above.
In this case, we have to provide, instead of `F`, the array `E` of shape (#edges, 2) containing in each row the indices of the two vertices connected by the edge.
Such array can be computed with [Libigl](https://libigl.github.io/tutorial/) as `E = igl.edges(F)`.

More generally, we can sum values at vertices from incident *n*-cells (if *n* = 2 the cell is an edge, if *n* = 3 a triange, if *n* = 4 a quadrilateral or a tetrahedron depending on the data structure, and so on).
We can define a generic function that works for *n*-cells and that can perform sums both per-cell and per-cell-vertex as follows:

In [18]:
def vertex_cells_sum(values, cells):
    """
    Sums values at vertices from each incident n-cell.

    Input:
    - values : np.array (#cells,) or (#cells, n)
        The cell values to be summed at vertices.
        If shape (#cells,): The value is per-cell,
        If shape (#cells, n): The value is per-cell-vertex.
        The value (i, j) corresponds to cell vertex (i, j)
    - cells : np.array (#cells, n)
        The array of cells.
    Output:
    - v_sum : np.array (#vertices,)
        A vector with the sum at each i-th vertex in i-th position.
    Note:
        If n = 2 the cell is an edge, if n = 3 the cell is a triangle,
        if n = 4 the cell can be a tetrahedron or a quadrilateral,
        depending on the data structure.
    """
    i = cells.flatten('F')
    j = np.arange(len(cells))
    j = np.tile(j, cells.shape[1])
    v = values.flatten('F')
    if len(v) == len(cells): # tests if values are per-cell.
        v = v[j]
    v_sum = sparse.coo_matrix((v, (i, j)), (np.max(cells) + 1, len(cells)))
    return np.array(v_sum.sum(axis=1)).flatten()